# Fantano Review Notebook 1 - Exploratory Data Analysis

*In this project*, I will analyze and visualize data of albums rated by "The Internet's Busiest Music Nerd," Anthony Fantano. Fantano's YouTube channel "The Needle Drop" has 2.12 Million subscribers (as of April 2020) and he has rated over 1,700 albumbs. After analyzing and visualizing the data, I will make two Machine Learning models to predict future album ratings. The first model will be a Logistic Regession model for binary classification which will classify albums as either "highly rated" or "not highly rated." The second will be a Neural Network which will classify albums on the same 1-10 scale that Fantano uses for all of his ratings.  

*In this first notebook*, I will combine two CSV files (one with Anthony Fantano ratings information from Kaggle and one with Spotify album information created using a Spotify API) to explore data of albums that Anthony Fantano has rated. I will combine the CSVs, clean the data, and organize the final CSV is a way that is conducive to visualization and modeling.